In [1]:
import xarray as xr
import numpy as np

In [2]:
data = xr.DataArray(np.random.randn(2, 3), dims=("x", "y"), coords={"x": [10, 20]})
data

<xarray.DataArray (x: 2, y: 3)>
array([[-0.25125629, -0.64688177, -2.60199958],
       [-2.24310876,  0.26687764,  2.31825927]])
Coordinates:
  * x        (x) int64 10 20
Dimensions without coordinates: y

In [3]:
data.values

array([[-0.25125629, -0.64688177, -2.60199958],
       [-2.24310876,  0.26687764,  2.31825927]])

In [4]:
data.dims

('x', 'y')

In [5]:
data.coords

Coordinates:
  * x        (x) int64 10 20

In [6]:
data[0, :]

<xarray.DataArray (y: 3)>
array([-0.25125629, -0.64688177, -2.60199958])
Coordinates:
    x        int64 10
Dimensions without coordinates: y

In [7]:
data.loc[10]

<xarray.DataArray (y: 3)>
array([-0.25125629, -0.64688177, -2.60199958])
Coordinates:
    x        int64 10
Dimensions without coordinates: y

In [8]:
data.isel(x=0)

<xarray.DataArray (y: 3)>
array([-0.25125629, -0.64688177, -2.60199958])
Coordinates:
    x        int64 10
Dimensions without coordinates: y

In [9]:
data.sel(x=10)

<xarray.DataArray (y: 3)>
array([-0.25125629, -0.64688177, -2.60199958])
Coordinates:
    x        int64 10
Dimensions without coordinates: y

In [11]:
data.attrs["long_name"] = "random velocity"
data.attrs["units"] = "metres/sec"
data.attrs["description"] = "A random variable created as an example."


In [17]:
data.x.attrs["units"] = "x units"
data.x

<xarray.DataArray 'x' (x: 2)>
array([10, 20])
Coordinates:
  * x        (x) int64 10 20
Attributes:
    units:    x units

In [16]:
data += 100
data

<xarray.DataArray (x: 2, y: 3)>
array([[299.74874371, 299.35311823, 297.39800042],
       [297.75689124, 300.26687764, 302.31825927]])
Coordinates:
  * x        (x) int64 10 20
Dimensions without coordinates: y
Attributes:
    long_name:    random velocity
    units:        metres/sec
    description:  A random variable created as an example.

In [14]:
data.T


<xarray.DataArray (y: 3, x: 2)>
array([[199.74874371, 197.75689124],
       [199.35311823, 200.26687764],
       [197.39800042, 202.31825927]])
Coordinates:
  * x        (x) int64 10 20
Dimensions without coordinates: y
Attributes:
    long_name:    random velocity
    units:        metres/sec
    description:  A random variable created as an example.

In [19]:
a = xr.DataArray(np.random.randn(3), [data.coords["y"]])
a

<xarray.DataArray (y: 3)>
array([-0.1722065 , -0.21636916,  0.26668037])
Coordinates:
  * y        (y) int64 0 1 2

In [21]:
b = xr.DataArray(np.random.randn(4), dims="z")
b

<xarray.DataArray (z: 4)>
array([-0.27844081, -0.48096555,  0.96710649, -0.98120523])
Dimensions without coordinates: z

In [25]:
c = a + b
c

<xarray.DataArray (y: 3, z: 4)>
array([[-0.45064731, -0.65317205,  0.7949    , -1.15341173],
       [-0.49480998, -0.69733471,  0.75073733, -1.1975744 ],
       [-0.01176044, -0.21428518,  1.23378686, -0.71452487]])
Coordinates:
  * y        (y) int64 0 1 2
Dimensions without coordinates: z

In [27]:
c.

<xarray.DataArray 'z' (z: 4)>
array([0, 1, 2, 3])
Dimensions without coordinates: z